**Collect images using opencv**

In [ ]:
from google.colab import output
from base64 import b64decode
from PIL import Image
import io, os, uuid, time, json

# Config
SAVE_DIR = "data/images"
os.makedirs(SAVE_DIR, exist_ok=True)

number_images = 30
interval = 0.5
quality = 0.8

# JS: capture N frames spaced by 'interval' seconds and return JSON array of dataURLs
js = f"""
new Promise(async (resolve, reject) => {{
  try {{
    const N = {number_images};
    const interval = {interval} * 1000;
    const quality = {quality};

    // Create video element
    const video = document.createElement('video');
    video.style.display = 'block';
    video.width = 640;
    document.body.appendChild(video);

    // Request camera
    const stream = await navigator.mediaDevices.getUserMedia({{video: true}});
    video.srcObject = stream;
    await video.play();

    // Wait a little for camera auto-expposure/ auto-focus to stabilize
    await new Promise(r => setTimeout(r, 500));

    const results = [];
    // Create canvas once
    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth || 640;
    canvas.height = video.videoHeight || 480;
    const ctx = canvas.getContext('2d');

    for (let i=0; i<N; i++) {{
      // draw current frame
      ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

      // get dataURL
      const dataUrl = canvas.toDataURL('image/jpeg', quality);
      results.push(dataUrl);

      // wait interval unless last
      if (i < N - 1) {{
        await new Promise(r => setTimeout(r, interval));
      }}
    }}

    // stop camera and cleanup
    stream.getTracks().forEach(t => t.stop());
    video.remove();

    // return JSON string
    resolve(JSON.stringify(results));
  }} catch (err) {{
    reject("ERROR_ACCESS_CAMERA:" + err.toString());
  }}
}});
"""

print("Requesting camera... allow permission in the browser when prompted.")
data = output.eval_js(js)

if isinstance(data, str) and data.startswith("ERROR_ACCESS_CAMERA:"):
  raise RuntimeError(data)

# Parse returnd JSON array of dataURLs
dataurls = json.loads(data)

print(f"Received {len(dataurls)} images; saving to {SAVE_DIR} ...")
saved = []
for i, d in enumerate(dataurls):
  header, encoded = d.split(',', 1)
  img_bytes = b64decode(encoded)
  img = Image.open(io.BytesIO(img_bytes))
  filename = os.path.join(SAVE_DIR, f"{str(uuid.uuid1())}.jpg")
  img.save(filename)
  saved.append(filename)
  time.sleep(0.5)

print("Done. Saved files:")
for p in saved:
  print(" -", p)